# Take Home Assignment #1

## Task #1

#### Integrate information from the “job_url_data” folder into one dictionary. The keys of the dictionary are individual urls scraped from the website, and the values are the earliest date that the corresponding urls were scraped. Save the results into a json file. How many unique job urls have been collected between May 17, 2022 and May 23, 2022?

In [2]:
###Preparation
import pandas as pd
import numpy as np
import os
os.chdir('/Users/[editted]/Desktop/SOCDATA/Take_home_1/indeed_scraped_data/job_url_data')
import glob
import csv
import json

###Adding column in dataset
with open('url_output.csv','w',newline='') as f_output:
    url_output=csv.writer(f_output)
    for name in glob.glob('*.csv'):
        if len(name)>15:
            with open(name,newline='') as inputs:
                url_input = csv.reader(inputs)
                for element in url_input:
                    element.insert(0,name[22:29]) # name[22:29]: where the date lies
                    url_output.writerow(element)

url_total = pd.read_csv('url_output.csv', low_memory=False, header=None)
url_total = url_total[url_total[1].str.contains("job_url") == False]

url_total# Checking the dataset
url_total = url_total.rename(columns={0: 'Date'})
url_total = url_total.rename(columns={1: 'job_url'})
url_total = url_total.groupby('job_url',as_index=False).min()

url_total = url_total.drop_duplicates()
url_total.shape
##Analysis: There are 21260 unique job urls

###Making doctionary
url_total_d = url_total
url_total_d.set_index('job_url',inplace=True)
url_d = url_total_d.to_dict()['Date']
url_d #Checking dictionary

###Converting to json
with open ('url_output.json','w') as json_f:
    json.dump(url_d,json_f)

## Task #2

#### Clean and integrate information from the “job_info_data” folder into one data frame. Files from this subfolder might have two different formats. Some of them are csv files, while others are json files. The columns might also be named differently. Find ways to read each of the files into pandas, drop records with missing job titles and/or missing job descriptions, and combine them into one dataframe. Lastly, drop records with duplicate job urls, and then save them into a separate csv file. How many unique jobs are there in the cleaned dataframe? 

In [9]:
os.chdir('/Users/[editted]/Desktop/SOCDATA/Take_home_1/indeed_scraped_data/job_info_data')

###Converting Jason
##json_1
json_1 = pd.read_json('parsehub_5182022_v2.json')
d_1 = pd.DataFrame.from_dict(json_1)
lnk_d1 = d_1['lnks'].apply(pd.Series)
##json_2
json_2 = pd.read_json('parsehub_5202022_v2.json')
d_2 = pd.DataFrame.from_dict(json_2)
lnk_d2 = d_2['lnks'].apply(pd.Series)
##Change column names, Check names, and store the dataset
lnk_d1 = lnk_d1.rename(columns={'link':'lnks_link','job_title':'lnks_job_title','company':'lnks_company','company_url':'lnks_company_url','company_location':'lnks_company_location','job_description':'lnks_job_description'})
print(lnk_d1.columns)
lnk_d1.to_csv('parsehub_5182022_v2.csv')
lnk_d2 = lnk_d2.rename(columns={'link':'lnks_link','job_title':'lnks_job_title','company':'lnks_company','company_url':'lnks_company_url','company_location':'lnks_company_location','job_description':'lnks_job_description'})
print(lnk_d2.columns)
lnk_d2.to_csv('parsehub_5202022_v2.csv')

###Merge all the data set in the info folder
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames
#combine all files in the list
combined_csv_info = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv_info.shape # Total of 43877 columns
# combined_csv_info

##Data Cleaning
#Drop NAs
combined_csv_info_new = combined_csv_info.dropna(subset=['lnks_job_title','lnks_job_description'])
##Drop Duplicates
combined_csv_info_final = combined_csv_info_new.drop_duplicates(subset=['lnks_link'])
combined_csv_info_final.shape

###There are 16303 unique job urls


###Write csv
combined_csv_info_final.to_csv( "combined_csv_info_final.csv", index=False, encoding='utf-8-sig')
# combined_csv_info_final.columns
combined_csv_info_final.shape

Index(['lnks_link', 'lnks_job_title', 'lnks_company', 'lnks_company_url',
       'lnks_company_location', 'lnks_job_description'],
      dtype='object')
Index(['lnks_link', 'lnks_job_title', 'lnks_company', 'lnks_company_url',
       'lnks_company_location', 'lnks_job_description'],
      dtype='object')


(16303, 7)

## Task #3

#### Merge between “job_url_data” and “job_info_data”. What is the percentage of jobs that can be matched between these two data sources? How are the missing data (unmatched job urls) distributed by date? What about matched job urls? How many complete job listings were we able to collect each day? How would you interpret this result with respect to data quality? Does it mean that our data collection strategy is flawed and thus introduces non-random sampling biases? 

In [8]:
###Merge Data Set
##Getting the data set
os.chdir('/Users/[editted]/Desktop/SOCDATA/Take_home_1/indeed_scraped_data/job_url_data')
##Open data sets
url_total = pd.read_csv('url_output.csv', low_memory=False, header=None)
url_total = url_total[url_total[1].str.contains("job_url") == False]
url_total = url_total.rename(columns={0: 'Date'})
url_total = url_total.rename(columns={1: 'job_url'})
url_total = url_total.groupby('job_url',as_index=False).min()

url_total = url_total.drop_duplicates()
url_total
##Checking columns
# print(url_total.columns)
# print(combined_csv_info_final.columns)

##Merge Data set
total_final = url_total.merge(combined_csv_info_final, left_on='job_url',right_on='lnks_link')
total_final = total_final.drop_duplicates()
total_final.shape #There are total 16303 data entries

###Percentage of merging successfully
#16303/(16303+21260)
#There are 43.4% files merged successfully 

###How many complete job listings were we able to collect each day?
total_final = total_final.dropna(subset=['lnks_job_title','lnks_job_description'])
#There are 16303 complete job listings that can be colleced, meaning no missing titles and descriptions.

###Daily Quality and Effectiveness of data collection process
date_d = {}

for element in total_final['Date']:
    if element not in date_d:
        date_d[element] = 1
    else:
        date_d[element] +=1
date_d
##Finding: i think the data quality is pretty correct because most of the jobs are 
#          found on 5/17, 5/18, 5/19 which are all week days, 
#          since there are more jobs appearing during week days than weekends

##Since this is a job mining project, I do not necesaarily think random selecting is required.




{5172022: 3905,
 5202022: 4503,
 5192022: 3790,
 5182022: 1324,
 5212022: 2113,
 5222022: 668}